## Генерация ответа с расширенным контекстом
Используем обогащённый контекст для генерации более точных ответов.

In [9]:
def generate_enriched_response(query, context, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    Генерирует ответ на основе расширенного контекста.
    
    Аргументы:
    query (str): Пользовательский запрос.
    context (str): Расширенный контекст.
    model (str): Модель для генерации ответа.
    
    Возвращает:
    str: Сгенерированный ответ.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Ты - AI ассистент, который строго отвечает на основе предоставленного контекста. Если ответа нет в контексте, скажи 'Недостаточно информации для ответа'."},
            {"role": "user", "content": f"Контекст:\n{context}\n\nВопрос: {query}"}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content

## Оценка качества ответов
Сравниваем качество ответов с обычным и обогащённым контекстом.

In [10]:
def evaluate_response(question, response, true_answer):
    """
    Оценивает качество ответа по двум критериям: достоверность и релевантность.
    
    Аргументы:
    question (str): Оригинальный вопрос.
    response (str): Оцениваемый ответ.
    true_answer (str): Эталонный ответ.
    
    Возвращает:
    dict: Оценки по шкале 0-1 для каждого критерия.
    """
    # Промт для оценки достоверности
    faithfulness_prompt = f"""
    Оцените достоверность ответа по сравнению с эталоном:
    Вопрос: {question}
    Ответ: {response}
    Эталон: {true_answer}
    
    Верните только число от 0 до 1, где:
    1 - полностью соответствует эталону
    0.5 - частично соответствует
    0 - не соответствует
    """
    
    # Промт для оценки релевантности
    relevancy_prompt = f"""
    Оцените релевантность ответа вопросу:
    Вопрос: {question}
    Ответ: {response}
    
    Верните только число от 0 до 1, где:
    1 - полностью отвечает на вопрос
    0.5 - частично отвечает
    0 - не отвечает
    """
    
    # Получаем оценки
    faithfulness = float(client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[{"role": "user", "content": faithfulness_prompt}],
        temperature=0
    ).choices[0].message.content)
    
    relevancy = float(client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[{"role": "user", "content": relevancy_prompt}],
        temperature=0
    ).choices[0].message.content)
    
    return {"faithfulness": faithfulness, "relevancy": relevancy}

## Пример полного цикла работы
Демонстрация работы контекстно-обогащённого RAG на реальном запросе.

In [11]:
def run_full_pipeline(query, documents, document_embeddings):
    """
    Выполняет полный цикл контекстно-обогащённого RAG.
    
    Аргументы:
    query (str): Пользовательский запрос.
    documents (List[dict]): Список документов.
    document_embeddings (List[np.ndarray]): Эмбеддинги документов.
    
    Возвращает:
    dict: Результаты с базовым и улучшенным ответами.
    """
    # 1. Находим релевантные документы
    relevant_docs = semantic_search(query, documents, document_embeddings)
    
    # 2. Извлекаем фрагменты из топ-1 документа
    chunks = extract_relevant_chunks(relevant_docs[0]['document'], query)
    
    # 3. Генерируем базовый ответ (без расширения контекста)
    basic_context = "\n".join([chunk['text'] for chunk in chunks])
    basic_response = generate_enriched_response(query, basic_context)
    
    # 4. Расширяем контекст
    enriched_context = expand_context(chunks, query)
    enriched_response = generate_enriched_response(query, enriched_context)
    
    return {
        "basic_response": basic_response,
        "enriched_response": enriched_response,
        "context_size": {
            "basic": len(basic_context.split()),
            "enriched": len(enriched_context.split())
        }
    }

## Запуск и оценка
Загружаем данные и запускаем полный цикл обработки.

In [12]:
# Загружаем документы
documents = load_documents("data")

# Создаём эмбеддинги
document_embeddings = create_document_embeddings(documents)

# Загружаем тестовые вопросы
with open('data/val.json') as f:
    test_questions = json.load(f)

# Выбираем первый вопрос для демонстрации
query = test_questions[0]['question']
true_answer = test_questions[0]['ideal_answer']

In [13]:
# Запускаем полный цикл
results = run_full_pipeline(query, documents, document_embeddings)

# Оцениваем результаты
basic_evaluation = evaluate_response(query, results["basic_response"], true_answer)
enriched_evaluation = evaluate_response(query, results["enriched_response"], true_answer)

## Вывод результатов
Сравниваем качество ответов с разными подходами.

In [14]:
print(f"Базовый ответ ({results['context_size']['basic']} слов):\n{results['basic_response']}\n")
print(f"Оценка: Достоверность={basic_evaluation['faithfulness']}, Релевантность={basic_evaluation['relevancy']}\n")

print(f"Обогащённый ответ ({results['context_size']['enriched']} слов):\n{results['enriched_response']}\n")
print(f"Оценка: Достоверность={enriched_evaluation['faithfulness']}, Релевантность={enriched_evaluation['relevancy']}")